#### #1 Verversen access token

In [1]:
import requests
from dotenv import load_dotenv
import os

load_dotenv('tokens.env')  # Laadt de .env file

client_id = os.getenv('STRAVA_CLIENT_ID')
client_secret = os.getenv('STRAVA_CLIENT_SECRET')
refresh_token = os.getenv('STRAVA_REFRESH_TOKEN')

def ververs_access_token(client_id, client_secret, refresh_token):
    url = 'https://www.strava.com/oauth/token'
    payload = {
        'client_id': client_id,
        'client_secret': client_secret,
        'grant_type': 'refresh_token',
        'refresh_token': refresh_token
    }
    response = requests.post(url, data=payload)
    if response.status_code == 200:
        tokens = response.json()
        # Sla nieuwe refresh_token op!
        new_refresh_token = tokens['refresh_token']
        access_token = tokens['access_token']
        
         
        print("Access token vernieuwd")
        return access_token, new_refresh_token
    else:
        print("Fout bij token vernieuwen:", response.status_code)
        print(response.text)
        return None, None

access_token, refresh_token = ververs_access_token(client_id, client_secret, refresh_token)
if not access_token:
    raise SystemExit("Kan geen toegang krijgen tot Strava API")


Access token vernieuwd


#### #2 Ophalen activiteiten

In [2]:
def haal_alle_activiteiten_op(access_token, per_pagina=200):
    url = 'https://www.strava.com/api/v3/athlete/activities'
    headers = {'Authorization': f'Bearer {access_token}'}
    alle_activiteiten = []
    pagina = 1

    while True:
        params = {'page': pagina, 'per_page': per_pagina}
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code != 200:
            print(f"Fout op pagina {pagina}: {response.status_code}")
            break
        
        data = response.json()
        if not data:  # Lege lijst = einde
            break
            
        alle_activiteiten.extend(data)
        print(f"Pagina {pagina}: {len(data)} activiteiten")
        pagina += 1

    print(f"Totaal: {len(alle_activiteiten)} activiteiten opgehaald")
    return alle_activiteiten

activiteiten = haal_alle_activiteiten_op(access_token)

Pagina 1: 47 activiteiten


Totaal: 47 activiteiten opgehaald


#### #3 dataframe maken

In [3]:
import pandas as pd

if not activiteiten:
    print("Geen activiteiten om op te slaan")
else:
    df = pd.DataFrame(activiteiten)
    
    # Extra kolommen
    df['distance_km'] = df['distance'] / 1000
    df['moving_time_min'] = df['moving_time'] / 60
    df['start_date_local'] = pd.to_datetime(df['start_date_local'])
    df['average_speed_kmu'] = df['average_speed'] * 3.6
    df['max_speed_kmu'] = df['max_speed'] * 3.6
    # Sla op
    csv_path = 'strava_activiteiten.csv'
    df.to_csv(csv_path, sep=';', index=False)
    print(f"{len(df)} activiteiten opgeslagen in {csv_path}")

47 activiteiten opgeslagen in strava_activiteiten.csv


#### #4 Logboek registratie

In [4]:
import datetime
import os

# Veilig logbestand buiten OneDrive → geen OneDrive-problemen meer
log_path = r"C:\temp\dashboard_log.txt"

# Maak de map C:\temp aan als die nog niet bestaat (één keer)
os.makedirs(r"C:\temp", exist_ok=True)

log_entry = f"[{datetime.datetime.now():%Y-%m-%d %H:%M:%S}] " \
            f"Script uitgevoerd - {len(activiteiten)} activiteiten, " \
            f"CSV bijgewerkt\n"

with open(log_path, "a", encoding="utf-8") as f:
    f.write(log_entry)

print(f"Log bijgewerkt: {log_path}")

Log bijgewerkt: C:\temp\dashboard_log.txt
